CODE TO EXPLORE AND MAYBE LOOK AT TESTING ACCURACY OF INITIAL DATA FROM SNOWFLAKE MARKETPLACE (FLIPSIDE CRYPTO) FOR POLYGON ONCHAIN DATA

In [20]:
#importing necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Load the dataset
data = pd.read_csv('../data/raw_data/initial_raw_data_polygon.csv')



In [21]:
#EDA of the dataset
for col in data.columns:
    print(f"\n--- Descriptive statistics for column: {col} ---")
    print(data[col].describe())


--- Descriptive statistics for column: WALLET ---
count                                          50000
unique                                         50000
top       0x429b8474bd7308b7787d364985bb4b8ea7de1d47
freq                                               1
Name: WALLET, dtype: object

--- Descriptive statistics for column: TX_PER_MONTH ---
count     50000.000000
mean        344.769023
std        6311.089609
min           0.166667
25%           1.000000
50%           8.333333
75%          24.666667
max      429229.833333
Name: TX_PER_MONTH, dtype: float64

--- Descriptive statistics for column: TOKEN_DIVERSITY ---
count    50000.000000
mean         6.308760
std         38.143845
min          0.000000
25%          0.000000
50%          1.000000
75%          3.000000
max       1631.000000
Name: TOKEN_DIVERSITY, dtype: float64

--- Descriptive statistics for column: PROTOCOL_DIVERSITY ---
count    5.000000e+04
mean     1.225921e+02
std      1.189325e+04
min      0.000000e+00
25%     